In [21]:
import numpy as np
import pandas as pd
from hazm import *

jhanr_dic = {
    'کلمات' : 0,
    'جامعه\u200cشناسی' : 1 ,
    'کلیات اسلام' : 2 ,
    'داستان کودک و نوجوانان' : 3, 
    'داستان کوتاه' : 4,
    'مدیریت و کسب و کار' : 5,
    'رمان' : 6
}

translator_jhanr = {
    0 : 'جامعه\u200cشناسی',
    1 : 'کلیات اسلام' ,
    2 : 'داستان کودک و نوجوانان', 
    3 : 'داستان کوتاه',
    4 : 'مدیریت و کسب و کار',
    5 : 'رمان'
}

words_dict = {}

def add_new_word(matris, word):
    matris[jhanr_dic['کلمات']].append(word)
    for i in range(1, 7):
        matris[i].append(0)
    return matris

def delete_sw(info):
    sw_pd = open("sw.csv", 'r')
    stop_words = sw_pd.read()
    stop_words = stop_words.split("\n")
    for i in range(0, len(stop_words)):
        while stop_words[i] in info:
            info.remove(stop_words[i])
    return info

# def stemmering(info):
#     stemmer = Stemmer()
#     for i in range(len(info)):
#             if not info[i].isnumeric():
#                 info[i] = stemmer.stem(info[i])
#     return info

def lemmetizing(info):
    lemmetizer = Lemmatizer()
    for i in range(len(info)):
            if not info[i].isnumeric():
                info[i] = lemmetizer.lemmatize(info[i])
    return info

def clean_txt(info):
    punctuaitions = ['(', '-', ')', '»', '«', '.', ':', '...', '،', '!', '؟', '/', '\\']
    for i in range(0, len(punctuaitions)):
        while punctuaitions[i] in info:
            info.remove(punctuaitions[i])
    info = delete_sw(info)
    info = lemmetizing(info)
    # info = stemmering(info)
    return info

books_train = pd.read_csv("books_train.csv")
books_test = pd.read_csv("books_test.csv")
normalizer = Normalizer()
splited_train_description = books_train['description'].apply(word_tokenize)
splited_train_description = splited_train_description.apply(clean_txt)
splited_test_description = word_tokenize(str(splited_train_description))
splited_test_description = books_test['description'].apply(word_tokenize)
splited_test_description = splited_test_description.apply(clean_txt)


In [22]:
train_bow = []
for i in range(0, 7):
    row = []
    train_bow.append(row)
for i in range(len(splited_train_description)):
    for j in range(len(splited_train_description[i])):
        if splited_train_description[i][j] in train_bow[jhanr_dic['کلمات']]:
            train_bow[jhanr_dic[books_train['categories'][i]]][words_dict[splited_train_description[i][j]][0]] += 1
        elif not splited_train_description[i][j].isnumeric():
            train_bow = add_new_word(train_bow, splited_train_description[i][j])
            words_dict[splited_train_description[i][j]] = []
            words_dict[splited_train_description[i][j]].append(len(words_dict) - 1)
            train_bow[jhanr_dic[books_train['categories'][i]]][words_dict[splited_train_description[i][j]][0]] += 1

In [23]:
def find_max_index(probabilities):
    max_index = 0
    for i in range(1, len(probabilities)):
        if probabilities[i] > probabilities[max_index]:
            max_index = i
    return max_index

def predict_jhanr_wt_ads(train_bow, description):
    probabilities = []
    for i in range(jhanr_dic['جامعه\u200cشناسی'], len(jhanr_dic)):
        makhrej = 0
        probability = 1
        for j in range(len(train_bow[i])):
            if str(train_bow[i][j]).isnumeric():
                makhrej += train_bow[i][j]
        for k in range(len(description)):
            if (description[k] in train_bow[jhanr_dic['کلمات']] and 
                str(train_bow[i][words_dict[description[k]][0]]).isnumeric()):
                probability *= (train_bow[i][words_dict[description[k]][0]] / makhrej)
        probabilities.append(probability)
    return translator_jhanr[find_max_index(probabilities)]

def accuracy_percentage(testing_data, given_data):
    num_of_currect = 0
    for i in range(len(testing_data)):
        if testing_data[i] == given_data[i]:
            num_of_currect += 1
    return num_of_currect / len(given_data)

prediction_result_without_ads = []
for i in range(len(splited_test_description)):
    prediction_result_without_ads.append(predict_jhanr_wt_ads(train_bow, splited_test_description[i]))
print("accuracy with out additive smooth is:")
print(accuracy_percentage(prediction_result_without_ads, books_test['categories']))


accuracy with out additive smooth is:
0.29555555555555557


In [24]:
def find_max_index(probabilities):
    max_index = 0
    for i in range(1, len(probabilities)):
        if probabilities[i] > probabilities[max_index]:
            max_index = i
    return max_index

def predict_jhanr_wt_ads(train_bow, description):
    probabilities = []
    a = 0.35
    for i in range(jhanr_dic['جامعه\u200cشناسی'], len(jhanr_dic)):
        N_Wi = 0
        probability = 0
        d = 0
        for j in range(len(train_bow[i])):
            if str(train_bow[i][j]).isnumeric():
                N_Wi += train_bow[i][j]
            if str(train_bow[i][j]).isnumeric() and train_bow[i][j] != 0:
                d += 1
        for k in range(len(description)):
            if (description[k] in train_bow[jhanr_dic['کلمات']] and 
                str(train_bow[i][words_dict[description[k]][0]]).isnumeric()):
                probability += np.log10((train_bow[i][words_dict[description[k]][0]] + a) / (N_Wi + a * d))
            elif not description[k].isnumeric():
                probability += np.log10((a) / (N_Wi + a * d))
        probabilities.append(probability)
    return translator_jhanr[find_max_index(probabilities)]

def accuracy_percentage(testing_data, given_data):
    num_of_currect = 0
    for i in range(len(testing_data)):
        if testing_data[i] == given_data[i]:
            num_of_currect += 1
    return num_of_currect / len(given_data)

prediction_result_without_ads = []
for i in range(len(splited_test_description)):
    prediction_result_without_ads.append(predict_jhanr_wt_ads(train_bow, splited_test_description[i]))
print("accuracy with additive smooth and log is:")
print(accuracy_percentage(prediction_result_without_ads, books_test['categories']))

accuracy with additive smooth and log is:
0.8111111111111111
